As starting point we take a SpatialData `.zarr` file, and preprocess it, so it can be visualized by Vitessce.

A SpatialData `.zarr` file can be created e.g. as follows:

```
import harpy as hp
sdata=hp.datasets.resolve_example()
sdata.write( ".../sdata_resolve.zarr" ) # change this output path
```

In [1]:
import os

from vitessce import (
    VitessceConfig,
    Component as cm,
    AnnDataWrapper,
    MultiImageWrapper,
)

from vitessce.data_utils import (
    optimize_adata,
    VAR_CHUNK_SIZE,
)

from vitessce.data_utils import multiplex_img_to_ome_zarr

In [ ]:
from spatialdata import read_zarr

path  = "/Users/arnedf/VIB/DATA/test_data" # change this path to where sdata is.

sdata = read_zarr( os.path.join( path, "sdata_resolve_vitessce" ) )

img_layer = "raw_image"
labels_layer = "segmentation_mask"
table_layer = "table_transcriptomics_cluster"

output_path_img= os.path.join( path, "raw_image.ome.zarr" )
output_path_masks= os.path.join( path, "masks.ome.zarr" )
output_path_adata= os.path.join( path, "adata.zarr" )

array=sdata[ "raw_image" ].data.compute()
multiplex_img_to_ome_zarr( img_arr=array, channel_names=[ "grey" ], axes="cyx",  output_path=output_path_img)
array=sdata[ "segmentation_mask" ].data.compute()
multiplex_img_to_ome_zarr( img_arr=array[ None,... ], channel_names=[ "grey" ], axes="cyx",  output_path=output_path_masks )

adata=sdata[ "table_transcriptomics_cluster" ]

adata.obs.index = adata.obs[ "cell_ID" ]
adata.obs.index.name = None

adata = optimize_adata(
    adata,
    obs_cols=[ "cell_ID", "total_counts", "leiden",],
    obsm_keys=[ "X_umap", "spatial"],
    optimize_X=True,
    to_dense_X=True,
)
adata.write_zarr(output_path_adata, chunks=[adata.shape[0], VAR_CHUNK_SIZE])

/Users/arnedf/miniconda3/envs/vitessce/lib/python3.12/site-packages/dask/dataframe/__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
/Users/arnedf/miniconda3/envs/vitessce/lib/python3.12/site-packages/zarr/creation.py:614: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
/Users/arnedf/miniconda3/envs/vitessce/lib/python3.12/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [3]:
sdata[ "table_transcriptomics_cluster" ].obs.head()

,cell_ID,fov_labels,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_2_genes,pct_counts_in_top_5_genes,n_counts,shapeSize,leiden
1,1,segmentation_mask,14,2.708050,38,3.663562,39.473684,65.789474,38,1063,4
2,2,segmentation_mask,11,2.484907,134,4.905275,76.865672,91.791045,134,2317,5
4,4,segmentation_mask,8,2.197225,136,4.919981,66.911765,88.970588,136,2192,1
5,5,segmentation_mask,10,2.397895,102,4.634729,79.411765,92.156863,102,1777,0
7,7,segmentation_mask,24,3.218876,53,3.988984,26.415094,47.169811,53,1417,4


In [4]:
from vitessce import OmeZarrWrapper

vc = VitessceConfig(schema_version="1.0.15", name='liver', description='Resolve liver')

dataset = vc.add_dataset(name='Liver dataset').add_object(
    MultiImageWrapper(
        image_wrappers=[
            OmeZarrWrapper(img_path=output_path_masks, is_bitmask=True, name='segmentations'),
            OmeZarrWrapper(img_path=output_path_img, is_bitmask=False, name='image'),
        ],
 )
)

dataset.add_object(AnnDataWrapper(
        adata_path=output_path_adata,
        obs_embedding_paths=["obsm/X_umap"],
        obs_embedding_names=["umap"],
        obs_locations_path="obsm/spatial",
        obs_set_paths=["obs/leiden"],
        obs_set_names=["Leiden"],
        obs_feature_matrix_path="X",
        coordination_values={
          "obsType": "cell",
          "featureType": "gene",
          "featureValueType": "expression"
        }
    ),
)

In [5]:
spatial_plot = vc.add_view(cm.SPATIAL, dataset=dataset)
layer_controller = vc.add_view(cm.LAYER_CONTROLLER, dataset=dataset)
umap_plot = vc.add_view(cm.SCATTERPLOT, dataset=dataset, mapping="umap")
cell_set_manager = vc.add_view(cm.OBS_SETS, dataset=dataset)
genes = vc.add_view(cm.FEATURE_LIST, dataset=dataset)

In [6]:
vc.layout((  ( spatial_plot | layer_controller ) / (  genes | cell_set_manager | umap_plot  ) ));

In [7]:
from IPython.display import display, HTML
url = vc.web_app()
display(HTML(f'<a href="{url}" target="_blank">Open in Vitessce</a>'))

In [8]:
import boto3

s3 = boto3.resource(
    service_name='s3',
    endpoint_url="https://objectstor.vib.be",
    aws_access_key_id=os.environ['ACCESS_KEY'],
    aws_secret_access_key=os.environ['SECRET_KEY'],
)

config_dict = vc.export(to='S3', s3=s3, bucket_name="spatial-hackathon-public", prefix='_test_vitessce_1')

Uploading spatial-hackathon-public:_test_vitessce_1/A/0/63f6a8a0-b563-46b8-9b51-03157ad5a543.ome.zarr
Uploading spatial-hackathon-public:_test_vitessce_1/A/0/e4be250e-c54e-44e8-ab0e-c2a22e7df01a.ome.zarr
Uploading spatial-hackathon-public:_test_vitessce_1/A/1/4565fb7e-fc28-4acc-9c97-363ae7f8efba.adata.zarr


In [9]:
import json

with open( "config_resolve.json", "w" ) as f: # this .json should be uploaded to the bucket, do not to forget to change url's before uploading
    json.dump(config_dict, f, indent=4)